- 多维的矩阵乘法细节

In [1]:
import torch

## 向量乘法

In [34]:
a1 @ a1

tensor(5)

In [2]:
a1 = torch.tensor([1, 2])
res1 = torch.matmul(a1, a1)
print(res1)
print(res1.shape)

tensor(5)
torch.Size([])


## 二维矩阵乘法

torch.mm 支持二维矩阵乘法

In [4]:
a2 = torch.tensor([[1, 2]])
print(a2.shape)

torch.Size([1, 2])


In [5]:
res2 = torch.matmul(a2, a2.transpose(-2, -1))
print(res2)
print(res2.shape)

tensor([[5]])
torch.Size([1, 1])


In [6]:
res2_1 = torch.mm(a2, a2.transpose(-2, -1))
print(res2_1)
print(res2_1.shape)

tensor([[5]])
torch.Size([1, 1])


torch.matmul 与 torch.mm 在二维矩阵运算上的结果一致

### 三维矩阵（高维矩阵乘法）

torch.bmm 支持三维矩阵乘法，不支持更高维度的矩阵乘法

高维的矩阵的乘法：

底层的两个维度的shape 需要满足做矩阵运算的条件。只在底层的两个维度上做矩阵乘法，矩阵高维度的shape不发生变化。

大家容易一听到矩阵乘法，都知道矩阵要做转置，对于二维矩阵乘法都很了解。
但对于高维矩阵乘法弄不清楚，不知道高维矩阵乘法是怎么在计算。

In [7]:
# 1. 做矩阵乘法的高维度通常都是一样的
a3 = torch.randn(2, 3, 2)
print(a3.shape)

torch.Size([2, 3, 2])


In [8]:
a3, a3.shape

(tensor([[[-0.3187, -2.1205],
          [-0.1837, -0.2859],
          [-0.5481, -1.2962]],
 
         [[ 0.0716,  1.0135],
          [-0.4479,  1.8286],
          [-0.4443, -0.3185]]]),
 torch.Size([2, 3, 2]))

In [9]:
a3.transpose(-1, -2), a3.transpose(-1, -2).shape

(tensor([[[-0.3187, -0.1837, -0.5481],
          [-2.1205, -0.2859, -1.2962]],
 
         [[ 0.0716, -0.4479, -0.4443],
          [ 1.0135,  1.8286, -0.3185]]]),
 torch.Size([2, 2, 3]))

In [10]:
res3 = torch.bmm(
    a3,
    a3.transpose(-1, -2)
)
print(res3)
print(res3.shape)

tensor([[[ 4.5979,  0.6648,  2.9231],
         [ 0.6648,  0.1155,  0.4713],
         [ 2.9231,  0.4713,  1.9805]],

        [[ 1.0323,  1.8212, -0.3546],
         [ 1.8212,  3.5445, -0.3834],
         [-0.3546, -0.3834,  0.2988]]])
torch.Size([2, 3, 3])


In [11]:
res3 = torch.matmul(
    a3,
    a3.transpose(-1, -2)
)
print(res3)
print(res3.shape)

tensor([[[ 4.5979,  0.6648,  2.9231],
         [ 0.6648,  0.1155,  0.4713],
         [ 2.9231,  0.4713,  1.9805]],

        [[ 1.0323,  1.8212, -0.3546],
         [ 1.8212,  3.5445, -0.3834],
         [-0.3546, -0.3834,  0.2988]]])
torch.Size([2, 3, 3])


a3 的 shape是(2, 3, 2)，a3 底层的两个维度做转置之后变成(2, 2, 3)，才可以做矩阵乘法。
可以发现第一位的数字都是2。

高维矩阵做乘法的时候，最底层两个维度要做足做矩阵乘法的条件，高维的shape两者都是一样的。如果不一致，需要是1，1可以做广播。



**高维矩阵乘法解读**：本质上是最后两个维度的矩阵计算。矩阵高维的shape不会发生变化。如果把最后两个维度的小矩阵抽象成为一个点的话，高维的矩阵乘法，本质上与向量乘法是一样的，都是把对应位置的点，直接相乘。

广播

In [46]:
t1 = torch.randn(1, 3, 2)
t2 = torch.randn(3, 2, 3)
t1 @ t2

tensor([[[-0.6557,  1.0518,  0.3055],
         [-0.2876, -2.5104, -1.4417],
         [ 1.4447, -0.1799,  0.4602]],

        [[ 0.2971,  0.0060, -0.2612],
         [-0.9089,  1.0824,  0.7131],
         [ 0.0929, -0.7898, -0.0199]],

        [[ 0.0027,  1.2031,  0.1543],
         [-0.5603, -1.8567, -0.1302],
         [ 0.3978, -0.9356, -0.1977]]])

In [41]:
torch.randn(1, 1, 2) @ torch.randn(3, 2, 3)

tensor([[[-0.3304, -1.4246, -1.4416]],

        [[-0.1239, -1.6294, -0.3421]],

        [[-2.3254, -1.6569, -1.6612]]])

In [48]:
torch.concat((t1, t1, t1)) @ t2

tensor([[[-0.6557,  1.0518,  0.3055],
         [-0.2876, -2.5104, -1.4417],
         [ 1.4447, -0.1799,  0.4602]],

        [[ 0.2971,  0.0060, -0.2612],
         [-0.9089,  1.0824,  0.7131],
         [ 0.0929, -0.7898, -0.0199]],

        [[ 0.0027,  1.2031,  0.1543],
         [-0.5603, -1.8567, -0.1302],
         [ 0.3978, -0.9356, -0.1977]]])

## 高维矩阵乘法

In [13]:
high_matrix1 = torch.randn(2, 3, 4, 5)

high_matrix2 = torch.randn(2, 3, 5, 4)

也可以使用 @ 做矩阵乘法

In [21]:
high_result = high_matrix1 @ high_matrix2

In [ ]:
torch.matmul(high_matrix1, high_matrix2) == (high_matrix1 @ high_matrix2)

把最后两个维度看成一个点。看两个矩阵对应位置的点相乘

shape(2, 3, 4, 5)与shape(2, 3, 5, 4)的矩阵相乘，若把最后两个维度看成一个点，

就可以类比为 (2, 3) 与 (2, 3)的两个矩阵做向量乘法，就是对应位置的点做乘法，不需要考虑shape的变换。

In [50]:
(high_matrix1[1][2] @  high_matrix2[1][2]) == high_result[1][2]

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])